In [1]:
import numpy as np
import pandas as pd

In [123]:
def hkl1(const):
    from math import sqrt
    
    for h in range(51):
        
        if h ** 2 > const:
            raise ValueError('h^2 > const; couldn\'t find an appropriate Miller plane')
        
        for k in range(51):
            
            if k ** 2 > const:
                break
            
            l = sqrt(const - h ** 2 - k ** 2)
            
            if l % 1 == 0:
                
                temp = [h, k, int(l)]
                temp.sort(reverse=True)
                
                return ('{' + str(temp[0]) + str(temp[1]) + str(temp[2]) + '}')

In [124]:
def hkl(arr):
    
    miller = []
    
    for i, const in enumerate(arr):
        miller.append(hkl1(const) )
    
    return miller

In [129]:
def analyse_diffraction(data, lamda, angle=2):
    
    decimals = len(str(lamda).split('.')[1])
    
    #first change data into a usable format
    if type(data) is str:
        
        if data.find('\n') != -1: #deal with data in separate lines
            data = data.splitlines()
            data = np.array([float(i.replace(',', '').strip()) for i in data])
            
        else: #deal with data separated by unknown numbers of spaces
            data = data.replace(',', '').split(' ')
            data = np.array(list(filter(lambda x: x != ' ', data) ))
    
    else:
        data = np.array(data)
    
    d = np.round(lamda / (2 * np.sin(np.radians(data / angle) ) ), decimals) #obtain d using Bragg equation
    cubicd = np.round(1 / d ** 2, decimals) #assume the unit cell is cubic
    const = np.round(cubicd / np.amin(cubicd), 3) #divide all values by the smallest to eliminate a
    
    #Names for dataframe columns
    name1 = r'$\frac{1}{d^{2}}$  /$\frac{1}{Å^2}$'
    name2 = r'$h^{2}$ ' + r' $k^{2}$ ' + r' $l^{2}$'
    
    #If h2 + k2 + l2 = integer, prcoeed as-is; if not, try to find an integer that will mutliple the const so that it
    ##becomes an integer
    if (const - np.round(const)  < 0.02).all():
        
        const = np.round(const).astype(np.uint16) #save const
        miller = hkl(const) #obtain miller planes
        
        return pd.DataFrame({'2θ': data, 'd /Å': d, name1: cubicd, name2: const, 'Index': miller})
        
    else:
        
        #look for an integer i such that i * const = integer
        for i in range(2, 101):
            
            const2 = const * i
            if (const2 - np.round(const2)  < 0.05).all():
                break
        
        const2 = np.round(const2).astype(np.uint16) #save const
        miller = hkl(const2) #obtain Miller planes
        
        return pd.DataFrame({'2θ': data, 'd /Å': d, name1: cubicd, 'const': const, name2: const2, 'Index': miller})

In [130]:
analyse_diffraction([27.38, 31.72, 45.47, 53.89, 56.50, 66.26, 75.33], 1.5418)

,2θ,d /Å,$\frac{1}{d^{2}}$ /$\frac{1}{Å^2}$,const,$h^{2}$ $k^{2}$ $l^{2}$,planes
0,27.38,3.2573,0.0943,1.000,3,{111}
1,31.72,2.8208,0.1257,1.333,4,{200}
2,45.47,1.9947,0.2513,2.665,8,{220}
3,53.89,1.7013,0.3455,3.664,11,{311}
4,56.50,1.6287,0.3770,3.998,12,{222}
5,66.26,1.4105,0.5026,5.330,16,{400}
6,75.33,1.2616,0.6283,6.663,20,{420}


In [131]:
analyse_diffraction([27.46,   39.22,   48.54,   56.68,   64.10,   71.09,   84.33], 1.5418)

,2θ,d /Å,$\frac{1}{d^{2}}$ /$\frac{1}{Å^2}$,$h^{2}$ $k^{2}$ $l^{2}$,planes
0,27.46,3.2480,0.0948,1,{100}
1,39.22,2.2970,0.1895,2,{110}
2,48.54,1.8755,0.2843,3,{111}
3,56.68,1.6240,0.3792,4,{200}
4,64.10,1.4527,0.4739,5,{210}
5,71.09,1.3261,0.5687,6,{211}
6,84.33,1.1484,0.7583,8,{220}


In [132]:
analyse_diffraction([22.783,   32.44,   40.01,   45.353,   52.419,   57.865,   67.925], 1.5418)

,2θ,d /Å,$\frac{1}{d^{2}}$ /$\frac{1}{Å^2}$,$h^{2}$ $k^{2}$ $l^{2}$,planes
0,22.783,3.9031,0.0656,1,{100}
1,32.440,2.7599,0.1313,2,{110}
2,40.010,2.2534,0.1969,3,{111}
3,45.353,1.9996,0.2501,4,{200}
4,52.419,1.7455,0.3282,5,{210}
5,57.865,1.5935,0.3938,6,{211}
6,67.925,1.3799,0.5252,8,{220}
